In [2]:

from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash

import pandas as pd
import numpy as np
import datetime as dt
import plotly.express as px
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objs as go


#### Import data for creating the energy consumption dashboard for the top 20 average energy consumer countries in the world

In [3]:
df_long = pd.read_csv('dataEnergyDashboard.csv')
display(df_long)

,Unnamed: 0,Country_Name,Country_Code,Year,Indicator_Name,Value
0,0,Kuwait,KWT,1990,Total Energy Use,4347.339585
1,1,Netherlands,NLD,1990,Total Energy Use,4394.542090
2,2,Czech Republic,CZE,1990,Total Energy Use,4796.941458
3,3,Belgium,BEL,1990,Total Energy Use,4809.867268
4,4,Palau,PLW,1990,Total Energy Use,4877.028215
...,...,...,...,...,...,...
1420,1420,Bahrain,BHR,2014,Energy Use from Renewables,0.000000
1421,1421,Trinidad and Tobago,TTO,2014,Energy Use from Renewables,49.539950
1422,1422,Iceland,ISL,2014,Energy Use from Renewables,13692.466657
1423,1423,Qatar,QAT,2014,Energy Use from Renewables,0.000000


#### Create Interactive Dashboard App Using Dash

In [4]:
app = dash.Dash()
df_long['Year'] = df_long['Year'].astype('int')

available_indicators = df_long['Indicator_Name'].unique()
available_countries = df_long['Country_Name'].unique()

app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Total Energy Use'
            )            
        ],
        style={'width': '50%', 'display': 'block'}),
        

    ], 
        style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'
    }),
    
    
    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-bar',
            hoverData={'points': [{'customdata': 'Canada'}]}
        )
    ], style={'width': '75%', 'display': 'block', 'padding': '20 20'}),

    html.Div(dcc.Slider(
        id='crossfilter-year--slider',
        min=df_long['Year'].min(),
        max=df_long['Year'].max(),
        value=df_long['Year'].max(),
        step=None,
        marks={str(year): str(year) for year in df_long['Year'].unique()}
    ), style={'width': '75%', 'padding': '0px 20px 20px 20px'}),
    
     html.Div([
        dcc.Graph(id='x-time-series')
    ], style={'display': 'block', 'width': '75%'})
])


@app.callback(
    dash.dependencies.Output('crossfilter-indicator-bar', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-year--slider', 'value')])

def update_graph(xaxis_column_name, 
                 year_value):
    dff = df_long[df_long['Year'] == year_value]

    return {
        'data': [go.Bar(
            x=dff[dff['Indicator_Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator_Name'] == xaxis_column_name]['Country_Name'],
            text=dff[dff['Indicator_Name'] == xaxis_column_name]['Country_Name'],
            customdata=dff[dff['Indicator_Name'] == xaxis_column_name]['Country_Name'],
            orientation='h',
#            mode='markers',
            marker=dict(
                color='rgba(0,0,128, 1)' if xaxis_column_name == 'Total Energy Use' else 'rgba(0,128,0,1)' if xaxis_column_name == 'Energy Use from Renewables' else 'rgba(255,0,0,1)'
            )
        )],
        'layout': go.Layout(
            xaxis={
                'title': xaxis_column_name,
            },
            yaxis={
                'title': 'Country',
            },
            margin={'l': 150, 'b': 30, 't': 10, 'r': 0},
            height=450,
            hovermode='closest'
        )
    }

def create_time_series(dff, title, xaxis_column_name):
    return {
        'data': [go.Scatter(
            x=dff['Year'],
            y=dff['Value'],
            mode='lines+markers',
             marker=dict(
                color='rgba(0,0,128, 1)' if xaxis_column_name == 'Total Energy Use' else 'rgba(0,128,0,1)' if xaxis_column_name == 'Energy Use from Renewables' else 'rgba(255,0,0,1)'
            )
        )],
        'layout': {
            'height': 400,
            'margin': {'l': 100, 'b': 30, 'r': 10, 't': 10},
            'annotations': [{
                'x': 0, 'y': 0.85, 'xanchor': 'left', 'yanchor': 'bottom',
                'xref': 'paper', 'yref': 'paper', 'showarrow': False,
                'align': 'left', 'bgcolor': 'rgba(255, 255, 255, 0.5)',
                'text': title
            }],
            'yaxis': {'type': 'linear', 'title': 'Energy Use (kg of oil equivalent per capita)'},
            'xaxis': {'showgrid': False, 'title': 'Year'}
        }
    }

@app.callback(
    dash.dependencies.Output('x-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-bar', 'hoverData'),
     dash.dependencies.Input('crossfilter-xaxis-column', 'value')])
def update_x_timeseries(hoverData, xaxis_column_name):
    country_name = hoverData['points'][0]['customdata']
    dff = df_long[df_long['Country_Name'] == country_name]
    dff = dff[dff['Indicator_Name'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, title, xaxis_column_name)

app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})

if __name__ == '__main__':
    app.run_server()


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
C:\Users\radhi\Documents\GitProjects\World_ESG\dash_Energy\myEnv\lib\site-packages\dash\resources.py:63: UserWarning: You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plot.com/external-resources
  warnings.warn(
127.0.0.1 - - [12/Jan/2022 12:00:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2022 12:00:28] "GET /_dash-component-suites/dash_renderer/react@16.v1_9_1m1617900226.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2022 12:00:28] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_9_1m1617900226.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2022 12:00:28] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_9_1m1617900226.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [12/